In [1]:
import os
from typing import Optional
from openai import OpenAI
from dotenv import load_dotenv
from abc import ABC, abstractmethod
load_dotenv()

client = OpenAI()

class Agent(ABC):
    
    def __init__(self, 
        model: Optional[str] = None, 
        system_prompt: Optional[str] = None
    ) -> None:
        
        model = model if model else os.getenv("OPENAI_DEFAULT_MODEL")
        
        self.memory = []
        self.model = model
        
        if system_prompt:
            self.memory.append({"role": "system", "content": system_prompt})
    
    def add_to_memory(self, role, message):
        self.memory.append({"role": role, "content": message})
    
    @abstractmethod
    def run(self,prompt):
       """Method that must be implemented by subclasses. Executes the agent's main functionality."""
    
class ConversationAgent(Agent):
    
    def __init__(self, 
        model: Optional[str] = None, 
        system_prompt: Optional[str] = None
    ) -> None:
        super().__init__(model,system_prompt)
    
    def run(self,prompt):

        self.add_to_memory("user", prompt)
        
        chat_completion = client.chat.completions.create(
            messages=self.memory,
            model=self.model,
        )
        response=chat_completion.choices[0].message.content
        
        self.add_to_memory("assistant", response)
        
        return response
    

In [2]:
#Define the agents
Emotion = ConversationAgent(model="gpt-3.5-turbo", system_prompt="You are a emocional expert. Your abstract text receipt analyzes feelings. Think step by step")
Facts = ConversationAgent(model="gpt-3.5-turbo", system_prompt="You are an expert in presidential speeches. Analyse the speech. Think step by step")
Helper = ConversationAgent(model="gpt-4", system_prompt="You receive a [SPEECH] from the user. Use [Emotion] and [Analysis] information to improve the user's speech [SPEECH]. Response only with the new improve speech")

# Define una solicitud de usuario más detallada para una tarea específica.
user_request = """
Mis queridos ¿cómo están?, bueno, eso no importa mucho en este momento. Bueno, digamos que estamos aquí porque toca hablar, aunque en realidad hay cosas más interesantes que podría estar haciendo.

Vamos a hablar sobre el estado de nuestro país, que, digamos, va bien o mal, depende de cómo lo mires. Hay cosas que se supone que debo decir, pero, sinceramente, ni yo mismo creo en todas ellas. Se nos viene un año bastante regular, pero no se preocupen, no es tan distinto de todos los otros años, ¿no?

Tenemos unos planes para mejorar las cosas, aunque, entre nosotros, sabemos que la mayoría de esos planes no van a llegar a ningún lado. Y sí, hay problemas, vaya que los hay, pero no son tan graves si te acostumbras, o si no piensas mucho en ellos.

Sabemos que hay gente que está teniendo problemas con el empleo, la salud y la educación, pero, eh, siempre ha sido así, ¿verdad? Estamos trabajando en ello, al menos eso es lo que el papel aquí dice. Tenemos proyectos para... bueno, para muchas cosas. A veces me pierdo con tantos nombres y estadísticas. Lo importante es que suena como que estamos haciendo algo.

Habrá quienes digan que deberíamos preocuparnos por el medio ambiente o algo así, pero bueno, esos temas no son tan urgentes. Necesitamos enfocarnos en lo inmediato, que es... ah, sí, asegurarnos de que se mantenga todo más o menos como está.

No olviden votarnos en las próximas elecciones, porque realmente necesitamos seguir aquí para... mantener las cosas bajo control, supongo. Y recuerden, no importa lo que pase, les diré que todo es parte del plan.

Gracias por escuchar, o no, da lo mismo.
"""

# El UserAgent recoge la solicitud y pide clarificaciones si son necesarias.
emotion_response = Emotion.run(user_request)
facts_response = Facts.run(user_request)
helper_response = Helper.run(f'[SPEECH]\n{user_request}\n\n\n[Emotion]\n{emotion_response}\n\n\n[Analysis]\n{facts_response}')

# Imprime la interacción completa
print(f"---> User:\n\n{user_request}\n")
print(f"---> emotion_response:\n\n{emotion_response}\n")
print(f"---> facts_response:\n\n{facts_response}\n")
print(f"---> helper_response:\n\n{helper_response}\n")



---> User:


Mis queridos ¿cómo están?, bueno, eso no importa mucho en este momento. Bueno, digamos que estamos aquí porque toca hablar, aunque en realidad hay cosas más interesantes que podría estar haciendo.

Vamos a hablar sobre el estado de nuestro país, que, digamos, va bien o mal, depende de cómo lo mires. Hay cosas que se supone que debo decir, pero, sinceramente, ni yo mismo creo en todas ellas. Se nos viene un año bastante regular, pero no se preocupen, no es tan distinto de todos los otros años, ¿no?

Tenemos unos planes para mejorar las cosas, aunque, entre nosotros, sabemos que la mayoría de esos planes no van a llegar a ningún lado. Y sí, hay problemas, vaya que los hay, pero no son tan graves si te acostumbras, o si no piensas mucho en ellos.

Sabemos que hay gente que está teniendo problemas con el empleo, la salud y la educación, pero, eh, siempre ha sido así, ¿verdad? Estamos trabajando en ello, al menos eso es lo que el papel aquí dice. Tenemos proyectos para... bueno